In [1]:
# STREAMLIT APP — SWIGGY RESTAURANT RECOMMENDER

import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px

# --------------------------------------------------
# PAGE CONFIG
# --------------------------------------------------
st.set_page_config(
    page_title="Swiggy Restaurant Recommender",
    page_icon="🍽️",
    layout="wide"
)

# --------------------------------------------------
# CUSTOM CSS
# --------------------------------------------------
st.markdown("""
<style>
.main-header {
    font-size: 3rem;
    color: #FF5722;
    text-align: center;
    font-weight: bold;
}
.sub-header {
    font-size: 1.5rem;
    color: #FF5722;
    margin-top: 2rem;
}
.card {
    background-color: #f0f2f6;
    padding: 1rem;
    border-radius: 10px;
    border-left: 5px solid #FF5722;
    margin-bottom: 1rem;
}
</style>
""", unsafe_allow_html=True)

# --------------------------------------------------
# DATA LOADING
# --------------------------------------------------
@st.cache_data
def load_data():
    cleaned = pd.read_csv(
        r"D:\Py_start\Python\project_SN\Project4\1\swiggy_cleaned_data.csv"
    )
    encoded = pd.read_csv(
        r"D:\Py_start\Python\project_SN\Project4\1\swiggy_encoded_data.csv"
    )
    encoded.index = cleaned.index
    return cleaned, encoded


@st.cache_resource
def load_models():
    with open(
        r"D:\Py_start\Python\project_SN\Project4\models\preprocessing.pkl", "rb"
    ) as f:
        return pickle.load(f)

# --------------------------------------------------
# RECOMMENDATION FUNCTIONS
# --------------------------------------------------
def get_similar_restaurants(df_cleaned, df_encoded, index, n=5):
    """Compute similarity ONLY for one restaurant"""
    base_vector = df_encoded.iloc[index].values.reshape(1, -1)
    scores = cosine_similarity(base_vector, df_encoded)[0]

    top_idx = np.argsort(scores)[::-1][1:n+1]
    result = df_cleaned.iloc[top_idx].copy()
    result["similarity"] = scores[top_idx]
    return result


def filter_restaurants(df, city, cuisine, min_rating, max_cost, n):
    if city != "All":
        df = df[df["city"] == city]
    if cuisine != "All":
        df = df[df["cuisine"].str.contains(cuisine, case=False, na=False)]
    df = df[df["rating"] >= min_rating]
    df = df[df["cost"] <= max_cost]
    return df.sort_values("rating", ascending=False).head(n)


def restaurant_card(row, show_similarity=False):
    st.markdown(f"""
    <div class="card">
        <h4>🍽️ {row['name']}</h4>
        📍 {row['city']}<br>
        🍴 {row['cuisine']}<br>
        ⭐ {row['rating']} | 💰 ₹{int(row['cost'])}
        {f"<br>🎯 Match: {row['similarity']:.2%}" if show_similarity else ""}
    </div>
    """, unsafe_allow_html=True)

# --------------------------------------------------
# MAIN APP
# --------------------------------------------------
def main():
    st.markdown('<p class="main-header">🍽️ Swiggy Restaurant Recommender</p>',
                unsafe_allow_html=True)

    try:
        df_cleaned, df_encoded = load_data()
        load_models()
    except Exception as e:
        st.error(f"❌ Data load failed: {e}")
        return

    st.sidebar.title("🎯 Choose Mode")
    mode = st.sidebar.radio(
        "Recommendation Type",
        ["🔍 Filter Search", "🎲 Similar Restaurants", "📊 Insights"]
    )

    # --------------------------------------------------
    # FILTER SEARCH
    # --------------------------------------------------
    if mode == "🔍 Filter Search":
        st.markdown('<p class="sub-header">🔍 Filter Restaurants</p>',
                    unsafe_allow_html=True)

        col1, col2 = st.columns(2)

        with col1:
            city = st.selectbox("City", ["All"] + sorted(df_cleaned.city.unique()))
            cuisine = st.selectbox("Cuisine", ["All"] + sorted(df_cleaned.cuisine.unique()))

        with col2:
            min_rating = st.slider("Min Rating", 0.0, 5.0, 3.5, 0.1)
            max_cost = st.slider(
                "Max Cost",
                50,
                int(df_cleaned.cost.max()),
                int(df_cleaned.cost.median()),
                50
            )

        n = st.slider("Results", 5, 20, 10)

        if st.button("🔍 Search"):
            results = filter_restaurants(
                df_cleaned, city, cuisine, min_rating, max_cost, n
            )
            for _, r in results.iterrows():
                restaurant_card(r)

    # --------------------------------------------------
    # SIMILARITY SEARCH
    # --------------------------------------------------
    elif mode == "🎲 Similar Restaurants":
        st.markdown('<p class="sub-header">🎲 Similar Restaurants</p>',
                    unsafe_allow_html=True)

        query = st.text_input("Search restaurant name")

        if query:
            matches = df_cleaned[
                df_cleaned["name"].str.contains(query, case=False, na=False)
            ]

            if not matches.empty:
                idx = st.selectbox(
                    "Select Restaurant",
                    matches.index,
                    format_func=lambda x: f"{df_cleaned.loc[x,'name']} ({df_cleaned.loc[x,'city']})"
                )

                n = st.slider("Similar Results", 3, 10, 5)

                if st.button("Find Similar"):
                    st.subheader("🍽️ Selected")
                    restaurant_card(df_cleaned.loc[idx])

                    st.subheader("🎯 Similar Restaurants")
                    recs = get_similar_restaurants(df_cleaned, df_encoded, idx, n)
                    for _, r in recs.iterrows():
                        restaurant_card(r, True)
            else:
                st.warning("No restaurant found")

    # --------------------------------------------------
    # INSIGHTS
    # --------------------------------------------------
    else:
        st.markdown('<p class="sub-header">📊 Dataset Insights</p>',
                    unsafe_allow_html=True)

        col1, col2, col3 = st.columns(3)
        col1.metric("Restaurants", len(df_cleaned))
        col2.metric("Cities", df_cleaned.city.nunique())
        col3.metric("Avg Rating", f"{df_cleaned.rating.mean():.2f}")

        fig = px.histogram(df_cleaned, x="rating", nbins=20,
                           title="Rating Distribution")
        st.plotly_chart(fig, use_container_width=True)


if __name__ == "__main__":
    main()


2026-01-31 22:38:24.736 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-31 22:38:24.737 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-31 22:38:25.268 
  command:

    streamlit run C:\Users\bro\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-01-31 22:38:25.269 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-31 22:38:25.269 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-31 22:38:25.270 No runtime found, using MemoryCacheStorageManager
2026-01-31 22:38:25.272 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-31 22:38: